<a href="https://colab.research.google.com/github/kanishk16/routetoMIDASS/blob/master/Python_Problem/Python_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>**Necessary Installations & Imports**

In [3]:
!pip install jsonlines

In [0]:
import tweepy
import json
import jsonlines

In [0]:
!wget -c https://github.com/wbolster/jsonlines/tree/1.2.0

--2019-04-06 18:58:10--  https://github.com/wbolster/jsonlines/tree/1.2.0
Resolving github.com (github.com)... 192.30.255.112, 192.30.255.113
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘1.2.0’

1.2.0                   [ <=>                ]  82.30K  --.-KB/s    in 0.01s   

2019-04-06 18:58:10 (5.48 MB/s) - ‘1.2.0’ saved [84279]



> **Task 1a : A python script to fetch all the tweets(as many as allowed by Twitter API) done by @midasIIITD twitter handle and dump the responses into JSONlines file**

> **Authorization for Twitter API**

In [0]:
"""
Stores tokens and keys
"""
consumer_key = "f69Ch2S4kVRsXbaZwhs0Qhc7M"
consumer_secret = "Z43Y85EwvKe3tOHVNAfvxEx7r7jmNliHhXDgZsdSnXuRIZKLKz"
access_key = "1034447027923374082-ftqEyERgMoJ8s8sMFICXfxy7UC44vs"
access_secret = "p8hg6CRuXVzn9yFxrnwObSCrm0gmW2PhKiveqw9SLwzsg"


In [0]:
# Authorization to consumer key and consumer secret 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 

# Access to user's access key and access secret 
auth.set_access_token(access_key, access_secret) 

# Calling api 
api = tweepy.API(auth) 

In [0]:
# api check
api

>**Tweet Extraction**

In [7]:
# tweets to be extracted 
tweets = api.user_timeline(screen_name='midasIIITD', number_of_tweets=10) 

# tweet check
print(type(tweets))
print(type(tweets[0]))
print(type(tweets[0]._json))

<class 'tweepy.models.ResultSet'>
<class 'tweepy.models.Status'>
<class 'dict'>




> ** Tweepy Status Object to dict to JSON string (ENCODING) to dict to list(DECODING)**




In [0]:
# tweepy status object to json object
tweet_x = [json.loads(json.dumps(tweet._json)) for tweet in tweets]

# tweet_x check
print(type(tweet_x))
print(type(tweet_x[1]))
print(type(json.dumps(tweets[0]._json)))

<class 'list'>
<class 'dict'>
<class 'str'>


>** dump into a jsonlines file**

In [0]:
# Open a JSON file to dump all the tweets
with open('/content/twitter12.jsonl', mode='w') as writer:
  for ii in tweet_x:
    json.dump(ii, writer)
    writer.write('\n')


> **Function to fetch all tweets and dump their responses into a JSONL file**

In [25]:
# Function to fetch all tweets and dump them into a jsonlines file 
def get_tweets(username): 
          
        # Authorization to consumer key and consumer secret 
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
  
        # Access to user's access key and access secret 
        auth.set_access_token(access_key, access_secret) 
  
        # Calling api 
        api = tweepy.API(auth) 
  
        # tweets to be extracted 
        tweets = api.user_timeline(screen_name=username, count=200, tweet_mode='extended')
        
        # tweepy status object to json object
        tweet_x = [json.loads(json.dumps(tweet._json)) for tweet in tweets] 
        
       # Open a JSON file to dump all the tweets
        with open('/content/twitter.jsonl', mode='w') as writer:
          for ii in tweet_x:
            json.dump(ii, writer)
            writer.write('\n')
            
        # Printing the tweets 
        print("RECORDED") 
  
  
if __name__ == '__main__': 
  
    # Twitter handle whose tweets are to be extracted 
    get_tweets("midasIIITD") 

RECORDED


### Task1b: Parse the tweets from the .jsonl file and convert them into a tabular format 


In [10]:
import pandas as pd

#tweet_store = []
with jsonlines.open('/content/twitter00.jsonl', mode='r') as reader:
  for r in reader:
    tweet_store = [json.loads(json.dumps(r)) for r in reader]
    
    # sanity check 
    print(type(tweet_store))
    print(type(tweet_store[0]))

df = pd.DataFrame(tweet_store, columns=['full_text','created_at','favorite_count','retweet_count'])    
 
print(df.head()) 

<class 'list'>
<class 'dict'>
                                           full_text  \
0  RT @IIITDelhi: Applications open for MTech (CB...   
1  RT @IIITDelhi: We are delighted to share that ...   
2  RT @Harvard: Professor Jelani Nelson founded A...   
3  RT @emnlp2019: For anyone interested in submit...   
4  RT @multimediaeval: Announcing the 2019 MediaE...   

                       created_at  favorite_count  retweet_count  
0  Wed Apr 10 04:51:26 +0000 2019               0              1  
1  Tue Apr 09 16:45:07 +0000 2019               0             14  
2  Tue Apr 09 05:04:27 +0000 2019               0             35  
3  Tue Apr 09 05:04:11 +0000 2019               0             17  
4  Mon Apr 08 19:38:09 +0000 2019               0             15  


>** Count and store the image_count in a list**

In [23]:
images = []
for tweet in tweet_store:
  image_count = 0
  try:
    if tweet['extended_entities']['media'][0]['type'] == 'photo':
      image_count += 1
  except:
    image_count = None
  images.append(image_count)

# sanity check
print(set(images))

{1, None}


>** Add the image_count column to the dataframe**

In [24]:
df['images_count'] = images

print(df)

                                             full_text  \
0    RT @IIITDelhi: Applications open for MTech (CB...   
1    RT @IIITDelhi: We are delighted to share that ...   
2    RT @Harvard: Professor Jelani Nelson founded A...   
3    RT @emnlp2019: For anyone interested in submit...   
4    RT @multimediaeval: Announcing the 2019 MediaE...   
5    Many Congratulations to @midasIIITD student, S...   
6    @midasIIITD thanks all students who have appea...   
7    @himanchalchandr Meanwhile, complete CV/NLP ta...   
8    @sayangdipto123 Submit as per the guideline ag...   
9    We request all students whose interview are sc...   
10   Other queries: "none of the Tweeter Apis give ...   
11   Other queries: "do we have to make two differe...   
12   Other queries: "If using Twitter api, it does ...   
13   Response to some queries asked by students on ...   
14   RT @kdnuggets: Top 8 #Free Must-Read #Books on...   
15   @nupur_baghel @PennDATS Congratulation @nupur_...   
16   We have e